In [1]:
import numpy as np
import scipy
from scipy import sparse
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn")
import seaborn as sns
import re
from tqdm import tqdm
import math
import nltk
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from bs4 import BeautifulSoup
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss, multilabel_confusion_matrix, confusion_matrix, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
import xml.etree.ElementTree as etree
import time
import spacy
from spacy import displacy
from wordcloud import WordCloud

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.min_rows', 50)

In [70]:
post = pd.read_csv('Data/Posts_Clean_1.csv', index_col=0, converters={'Tags_list' : str, 'Text' : str})
X = post["Text"].values
Y = post["Tags_list"].str.split().values
x_train, x_test, y_train, y_test = train_test_split(
  X, Y, test_size=0.2, random_state=9000)

In [71]:
tags_binarizer = MultiLabelBinarizer(sparse_output=True)
tags_binarizer.fit(post["Tags_list"].str.split())
y_train_binar = tags_binarizer.transform(post["Tags_list"].str.split())

## Supervisée

In [72]:
tags_binarizer = MultiLabelBinarizer()
tags_binarizer.fit(y_train)
y_train_binar = tags_binarizer.transform(y_train)
y_test_binar = tags_binarizer.transform(y_test)

count_vect = CountVectorizer(lowercase=False, token_pattern=r"\S+", max_features=10000)
count_vect.fit(x_train)
x_train_counts = count_vect.transform(x_train)
x_test_counts = count_vect.transform(x_test)